# NTRU Encrypt

In [12]:
d = 10

N = next_prime(1 << d)
p = 3
q = next_prime(p*N)

print d, q, p, N

10 3109 3 1031


In [13]:
Z.<x>  = ZZ[]        # polinómios de coeficientes inteiros
Q.<x>  = PolynomialRing(GF(q),name='x').quotient(x^N-1); Q

Univariate Quotient Polynomial Ring in x over Finite Field of size 3109 with modulus x^1031 + 3108

In [14]:
def vec():
    return  [choice([-1,0,1]) for k in range(N)]

# arredondamento módulo 'q'

def qrnd(f):    # argumento em 'Q'
    qq = (q-1)//2 ; ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

# arredondamento módulo 'p'

def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

# comparação módulo 'p'
#def equalp(x,y,p):
#    return  all([(a-b)%p == 0  for (a,b) in zip(x,y)] )

In [23]:
import hashlib

class NTRU(object):
    def __init__(self,N,p,q,d,seed):
        # calcular um 'f' invertível
        f = Q(0)
        while not f.is_unit():
            F = Q(vec()); f = 1 + p*F
        G = Q(vec()); g = p*G
        # gerar as chaves      #10
        h = g^(-1) * (p*f + 1)
        self.chvpriv = (p*f,g)
        self.chvpub = h

    def encrypt(self,msg,param,seed):
        #mlen = length(m)
        #param == N or p or q or d
        m = msg + seed

        sha256 = hashlib.sha256() #20
        sha256.update(m + self.chvpub)
        rseed = int(sha256.hexdigest(), 16)

        r = Q(vec())
        t = r*self.chvpub
        sha256.update(h)
        tseed = int(sha256.hexdigest(), 16)

        m_mask = Q(vec())
        m_2 = m - m_mask #30

        crip = t + m_2
        return crip


    def decrypt(self,crip,param):
        m_2 = Q(f * crip)
        t = crip - m
        sha256 = hashlib.sha256()
        sha256.update(h) #40
        tseed = int(sha256.hexdigest(), 16)

        m_mask = Q(vec())
        m = Q(m_2 + m_mask)

        sha256.update(m + self.chvpub)
        rseed = int(sha256.hexdigest(), 16)

        r = Q(vec())
        msg,mlen = m #50

        if p*r*self.chpub == t:
            result = msg,mlen
        else:
            result = "Error"
        return result

In [24]:
# Uma instância NTRU
K = NTRU(N,p,q,d,seed = vec())
# Uma mensagem aleatória
m = vec()
# Cifrar
e = K.encrypt(m,p,seed)  
# Decifrar e Verificar
m == K.decrypt(e,p)

TypeError: can only concatenate list (not "type") to list